<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.5: Putting it all Together: An FIR Filter
**Prev: [Sequential Logic](2.4_sequential_logic.ipynb)**<br>
**Next: [ChiselTest (was chisel-testers2)](2.6_chiseltest.ipynb)**

## Motivation
现在你已经学习了 Chisel 的基础知识，让我们利用这些知识构建一个 FIR（有限冲激响应）滤波器模块吧！FIR 滤波器在数字信号处理应用中非常常见。此外，FIR 滤波器将在模块3中频繁出现，因此请务必不要跳过这个模块！如果你不熟悉 FIR 滤波器，请查阅[可信的维基百科](https://en.wikipedia.org/wiki/Finite_impulse_response)上的文章以了解更多信息。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# FIR Filter

你将设计的 FIR 滤波器执行以下操作。

<img src="images/fir.jpg" width="720">

基本上，这进行滤波器系数元素与输入信号元素的逐元素乘法，并输出和（也称为_卷积_）。

或者，用信号的定义表示：

$y[n] = b_0 x[n] + b_1 x[n-1] + b_2 x[n-2] + ...$
 - $y[n]$ 是时间 $n$ 的输出信号
 - $x[n]$ 是输入信号
 - $b_i$ 是滤波器系数或脉冲响应
 - $n-1$, $n-2$, ... 是时间 $n$ 延迟 1, 2, ... 个周期
 
## 8-bit Specification

构建一个 4 元素 FIR 滤波器，其中四个滤波器系数是参数。为你提供了模块框架和基本测试。
注意，输入和输出都是 8 位无符号整数。你需要使用移位寄存器等构造来保存必要的状态（如延迟的信号值）。使用提供的测试器检查你的实现。
具有恒定输入的寄存器可以使用移位值为 1 的 `ShiftRegister` 创建，或使用 `RegNext` 构造。

注意：为了测试通过，你的寄存器必须初始化为 `0.U`。

In [ ]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  ???
}

In [ ]:
// Simple sanity check: a element with all zero coefficients should always produce zero
test(new My4ElementFir(0, 0, 0, 0)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.clock.step(1)
    c.io.in.poke(4.U)
    c.io.out.expect(0.U)
    c.clock.step(1)
    c.io.in.poke(5.U)
    c.io.out.expect(0.U)
    c.clock.step(1)
    c.io.in.poke(2.U)
    c.io.out.expect(0.U)
}

In [ ]:
// Simple 4-point moving average
test(new My4ElementFir(1, 1, 1, 1)) { c =>
    c.io.in.poke(1.U)
    c.io.out.expect(1.U)  // 1, 0, 0, 0
    c.clock.step(1)
    c.io.in.poke(4.U)
    c.io.out.expect(5.U)  // 4, 1, 0, 0
    c.clock.step(1)
    c.io.in.poke(3.U)
    c.io.out.expect(8.U)  // 3, 4, 1, 0
    c.clock.step(1)
    c.io.in.poke(2.U)
    c.io.out.expect(10.U)  // 2, 3, 4, 1
    c.clock.step(1)
    c.io.in.poke(7.U)
    c.io.out.expect(16.U)  // 7, 2, 3, 4
    c.clock.step(1)
    c.io.in.poke(0.U)
    c.io.out.expect(12.U)  // 0, 7, 2, 3
}

In [ ]:
// Nonsymmetric filter
test(new My4ElementFir(1, 2, 3, 4)) { c =>
    c.io.in.poke(1.U)
    c.io.out.expect(1.U)  // 1*1, 0*2, 0*3, 0*4
    c.clock.step(1)
    c.io.in.poke(4.U)
    c.io.out.expect(6.U)  // 4*1, 1*2, 0*3, 0*4
    c.clock.step(1)
    c.io.in.poke(3.U)
    c.io.out.expect(14.U)  // 3*1, 4*2, 1*3, 0*4
    c.clock.step(1)
    c.io.in.poke(2.U)
    c.io.out.expect(24.U)  // 2*1, 3*2, 4*3, 1*4
    c.clock.step(1)
    c.io.in.poke(7.U)
    c.io.out.expect(36.U)  // 7*1, 2*2, 3*3, 4*4
    c.clock.step(1)
    c.io.in.poke(0.U)
    c.io.out.expect(32.U)  // 0*1, 7*2, 2*3, 3*4
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + 
    x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + 
    x_n3 * b3.U(8.W)
</pre></article></div></section></div>

---
# FIR Filter Generator

在本模块中，我们将使用来自[Module 3.2: Generators: Collection](3.2_collections.ipynb)的一个稍作修改的示例。
如果你还没有开始模块 3.2，不用担心。
你将了解 `MyManyDynamicElementVecFir` 的工作原理，但基本思想是它是一个 FIR 滤波器生成器。

生成器有一个参数：长度。
该参数决定了滤波器有多少个抽头（taps），抽头是硬件 `Module` 的输入。

生成器有3个输入：
* in：滤波器的输入
* valid：一个布尔值，表示输入何时有效
* consts：一个包含所有抽头的向量

和1个输出：
* out：过滤后的输入

<img src="images/fir.jpg" style="width:450px;"/>

In [ ]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val valid = Input(Bool())
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })
  
  // Such concision! You'll learn what all this means later.
  val taps = Seq(io.in) ++ Seq.fill(io.consts.length - 1)(RegInit(0.U(8.W)))
  taps.zip(taps.tail).foreach { case (a, b) => when (io.valid) { b := a } }

  io.out := taps.zip(io.consts).map { case (a, b) => a * b }.reduce(_ + _)
}

visualize(() => new MyManyDynamicElementVecFir(4))

---
# DspBlock

将DSP组件集成到更大的系统中可能具有挑战性且容易出错。
[dsptools仓库的rocket部分](https://github.com/ucb-bar/dsptools/tree/master/rocket)包含一些有用的generators，可以帮助完成这些任务。

其中一个核心抽象是`DspBlock`的概念。
一个`DspBlock`具有：
* AXI-4 Stream输入和输出
* 内存映射的状态和控制（在这个例子中，是AXI4）

<img src="images/fir_filter.png" style="width:800px;"/>

`DspBlock`使用rocket中的diplomatic接口。
[这个网站](https://www.lowrisc.org/docs/diplomacy/)有一个关于diplomacy基础的很好的概述，但在这个例子中，不用太担心它是如何工作的。
当你连接许多不同的模块来形成一个复杂的SoC时，diplomacy的优势就显现出来了。
在这个例子中，我们只是制作一个单独的外设。
混入`StandaloneBlock` traits使diplomatic接口可以作为顶层IO工作。
只有当`DspBlock`作为顶层接口使用而没有任何diplomatic连接时，才需要它们。

以下代码将FIR滤波器封装在AXI4接口中。

In [ ]:
import dspblocks._
import freechips.rocketchip.amba.axi4._
import freechips.rocketchip.amba.axi4stream._
import freechips.rocketchip.config._
import freechips.rocketchip.diplomacy._
import freechips.rocketchip.regmapper._

//
// Base class for all FIRBlocks.
// This can be extended to make TileLink, AXI4, APB, AHB, etc. flavors of the FIR filter
//
abstract class FIRBlock[D, U, EO, EI, B <: Data](val nFilters: Int, val nTaps: Int)(implicit p: Parameters)
// HasCSR means that the memory interface will be using the RegMapper API to define status and control registers
extends DspBlock[D, U, EO, EI, B] with HasCSR {
    // diplomatic node for the streaming interface
    // identity node means the output and input are parameterized to be the same
    val streamNode = AXI4StreamIdentityNode()
    
    // define the what hardware will be elaborated
    lazy val module = new LazyModuleImp(this) {
        // get streaming input and output wires from diplomatic node
        val (in, _)  = streamNode.in(0)
        val (out, _) = streamNode.out(0)

        require(in.params.n >= nFilters,
                s"""AXI-4 Stream port must be big enough for all 
                   |the filters (need $nFilters,, only have ${in.params.n})""".stripMargin)

        // make registers to store taps
        val taps = Reg(Vec(nFilters, Vec(nTaps, UInt(8.W))))

        // memory map the taps, plus the first address is a read-only field that says how many filter lanes there are
        val mmap = Seq(
            RegField.r(64, nFilters.U, RegFieldDesc("nFilters", "Number of filter lanes"))
        ) ++ taps.flatMap(_.map(t => RegField(8, t, RegFieldDesc("tap", "Tap"))))

        // generate the hardware for the memory interface
        // in this class, regmap is abstract (unimplemented). mixing in something like AXI4HasCSR or TLHasCSR
        // will define regmap for the particular memory interface
        regmap(mmap.zipWithIndex.map({case (r, i) => i * 8 -> Seq(r)}): _*)

        // make the FIR lanes and connect inputs and taps
        val outs = for (i <- 0 until nFilters) yield {
            val fir = Module(new MyManyDynamicElementVecFir(nTaps))
            
            fir.io.in := in.bits.data((i+1)*8, i*8)
            fir.io.valid := in.valid && out.ready
            fir.io.consts := taps(i)            
            fir.io.out
        }

        val output = if (outs.length == 1) {
            outs.head
        } else {
            outs.reduce((x: UInt, y: UInt) => Cat(y, x))
        }

        out.bits.data := output
        in.ready  := out.ready
        out.valid := in.valid
    }
}

// make AXI4 flavor of FIRBlock
abstract class AXI4FIRBlock(nFilters: Int, nTaps: Int)(implicit p: Parameters) extends FIRBlock[AXI4MasterPortParameters, AXI4SlavePortParameters, AXI4EdgeParameters, AXI4EdgeParameters, AXI4Bundle](nFilters, nTaps) with AXI4DspBlock with AXI4HasCSR {
    override val mem = Some(AXI4RegisterNode(
        AddressSet(0x0, 0xffffL), beatBytes = 8
    ))
}

// running the code below will show what firrtl is generated
// note that LazyModules aren't really chisel modules- you need to call ".module" on them when invoking the chisel driver
// also note that AXI4StandaloneBlock is mixed in- if you forget it, you will get weird diplomacy errors because the memory
// interface expects a master and the streaming interface expects to be connected. AXI4StandaloneBlock will add top level IOs
// println(chisel3.Driver.emit(() => LazyModule(new AXI4FIRBlock(1, 8)(Parameters.empty) with AXI4StandaloneBlock).module))

## 测试

测试`DspBlock`有点不同。
现在我们涉及内存接口和`LazyModule`s。
dsptools具有一些功能，可以帮助测试`DspBlock`。

一个重要的功能是`MemMasterModel`。
该特性定义了诸如`memReadWord`和`memWriteWord`之类的函数 - 用于生成内存流量的通用函数。
这使您可以编写一个通用测试，可以针对您正在使用的内存接口进行特化 - 例如，您编写一个测试，然后针对TileLink和AXI4接口进行特化。

下面的代码以这种方式测试`FIRBlock`。

In [ ]:
import dsptools.tester.MemMasterModel
import freechips.rocketchip.amba.axi4

abstract class FIRBlockTester[D, U, EO, EI, B <: Data](c: FIRBlock[D, U, EO, EI, B]) extends PeekPokeTester(c.module) with MemMasterModel {
    // check that address 0 is the number of filters
    require(memReadWord(0) == c.nFilters)
    // write 1 to all the taps
    for (i <- 0 until c.nFilters * c.nTaps) {
        memWriteWord(8 + i * 8, 1)
    }
}

// specialize the generic tester for axi4
class AXI4FIRBlockTester(c: AXI4FIRBlock with AXI4StandaloneBlock) extends FIRBlockTester(c) with AXI4MasterModel {
    def memAXI = c.ioMem.get
}

// invoking testers on lazymodules is a little strange.
// note that the firblocktester takes a lazymodule, not a module (it calls .module in "extends PeekPokeTester()").
val lm = LazyModule(new AXI4FIRBlock(1, 8)(Parameters.empty) with AXI4StandaloneBlock)
chisel3.iotesters.Driver(() => lm.module) { _ => new AXI4FIRBlockTester(lm) }

<span style="color:red">**Exercise: TileLink**</span><br>

添加一个使用TileLink作为其内存互连的版本的`FIRBlock`，并扩展`FIRBlockTester`以使用TileLink。


---
# You're done!

[Return to the top.](#top)